## Análise de homologias por BLAST

1. [local_blastp(tag_to_files, db)](#local)
2. [docker_blastp(directory, db)](#docker)
3. [expasy_blastp(tag_to_files)](#expasy)
4. [blastp(tags_and_proteins, db, type)](#blast)
5. [write_queries_to_dir(tags_and_proteins, directory)](#write_queries_to_dir)
6. [extract_blast_info(tag_to_files, type)](#extract_blast_info)
7. [util.www.fetch_uniprots(ids)](#fetch_uniprots)
8. [util.www.extract_uniprot_info(entry)](#extract_uniprot_info)
9. [main()](#main)

O objectivo desta fase era, dadas as proteínas que retirámos do ficheiro genbank, procurar sequências homólogas com BLAST. Esta fase foi iterada várias vezes à procura da melhor estratégia, e no final acabámos com várias alternativas para a execução do BLAST, que podem ser encontradas no módulo [`util.blast`](https://github.com/vitorenesduarte/bionella/blob/master/util/blast.py). As alternativas são:

- __Local__
- __Docker__
- __ExPASy__

<a name="local"></a>
### Local

Esta função assume que o BLAST está instalado localmente, e recebe como parâmetro qual a base de dados que deve usar. O primeiro argumento desta função `tag_to_files` é um dicionário em que as chaves são __locus_tag__ e os valores são pares em que a primeira componente é o ficheiro onde se encontra a sequência de aminoácidos para ser usada na query (__in_file__), e a segunda componente é o ficheiro onde deve ser guardado o resultado do BLAST (__out_file__).

```python
def local_blastp(tag_to_files, db):
    """
    Corre o blast localmente.
    """

    for tag in tag_to_files:
        (in_file, out_file) = tag_to_files[tag]
    
        blastp_cline = NcbiblastpCommandline(
            query=in_file,
            db=db,
            evalue=10,
            outfmt=5,
            out=out_file
        )
        blastp_cline()
```

<a name="docker"></a>
### Docker

Para evitar a instalação de BLAST, download de bases de dados e sua adição como base de dados para ser usada pelo BLAST, criámos uma imagem __Docker__ com a base de dados __UniProtKB - Swiss-Prot__ [retirada do site da UniProt](http://www.uniprot.org/downloads). Mais informação sobre esta imagem pode ser encontrada [aqui](docker.html).

```python
def docker_blastp(directory, db):
    """
    Corre o blast numa instância docker.
    """
    cmd = "docker run -e QUERY_DIR=" + directory \
                  + " -e DB=" + db \
                  + " -v $PWD/" + directory + ":/" + directory \
                  + " -ti vitorenesduarte/swissprot_blast"

    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    p.wait()
```

<a name="expasy"></a>
### ExPASy

A terceira opção não corre o BLAST localmente, mas utiliza o serviço disponibilizado pelo site ExPASy.

```python
def expasy_blastp(tag_to_files):
    """
    Corre o blast na expasy.
    """
    for tag in tag_to_files:
        (in_file, out_file) = tag_to_files[tag]
        query = rw.read_file(in_file)
        blast_result = www.expasy_blast(query)
        rw.write_file(blast_result, out_file)
```

Esta função usa uma outra função definida no módulo [`util.www`](https://github.com/vitorenesduarte/bionella/blob/master/util/www.py) que corre o BLAST contra a base de dados __UniProtKB = Swiss-Prot + TrEMBL__, usando o serviço da ExPASy.

```python
def expasy_blast(query):
    """
    Corre o blast na expasy.
    """

    url = "http://web.expasy.org/cgi-bin/blast/blast.pl"
    data = {
        "format": "xml",
        "prot_db1": "UniProtKB",
        "matrix": "BLOSUM62",
        "showsc": 50, # best scoring sequences
        "showal": 50, # best alignments
        "seq": query
    }

    response = requests.post(
        url,
        data=data
    )

    return response.text
```

Esta última opção permitiu-nos encontrar sequências com maior homologia do que as que estávamos a encontrar ao usar apenas a __Swiss-Prot__ localmente.

<a name="blast"></a>
### BLAST

A função que engloba estas três opções recebe como argumento uma lista de pares em que a primeira componente é a __locus_tag__ da proteína, e a segunda componente é a respectiva sequência de aminoácidos. Recebe ainda qual o tipo de BLAST, __local__, __docker__ ou __expasy__, a utilizar.

```python
def blastp(tags_and_proteins, type="local"):
    """
    Corre o blast para a proteínas passadas como argumento.

    O argumento type é opcional e pode ter dois valores:
        - local
        - docker
        - expasy

    É retornado um dicionário com os resultados.
    """

    directory = ".query_dir"
    tag_to_files = write_queries_to_dir(tags_and_proteins, directory)

    # correr o blast
    if type == "local":
        local_blastp(tag_to_files, "swissprot")
    elif type == "docker":
        docker_blastp(directory, "swissprot")
    elif type == "expasy":
        expasy_blastp(tag_to_files)
    else:
        raise Exception("Unsupported type: " + type)

    blast_results = extract_blast_info(tag_to_files, type)
    return blast_results
```

Esta função usa duas outras funções:

<a name="write_queries_to_dir"></a>
- `write_queries_to_dir` que recebe uma lista de pares, __locus_tag__ e sequência de aminoácidos __protein__, e retorna um dicionário que mapeia __locus_tag__ para pares (__in_file__, __out_file__).

```python
def write_queries_to_dir(tags_and_proteins, directory):
    """
    Grava as proteínas passadas como argumento na
    diretoria também passada como argumento.
    """

    tag_to_files = {}

    # Apagar a diretoria e criar uma nova
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory)

    for (tag, protein) in tags_and_proteins:
        # Gravar a proteína num ficheiro
        in_file = directory + "/" + fasta_it(tag)
        rw.write_file(protein, in_file)

        # registar esta informação num dicionário
        out_file = directory + "/" + xml_it(fasta_it(tag))
        tag_to_files[tag] = (in_file, out_file)

    return tag_to_files
```

<a name="extract_blast_info"></a>
- `extract_blast_info` que dado este dicionário retorna um outro dicionário que mapeia __locus_tag__ para a lista de resultados do BLAST. Cada um destes resultados é um dicionário com:
    - __uniprot_id__
    - __evalue__
    - __score__
    - __identity__

```python
def extract_blast_info(tag_to_files, type):
    """
    Extrai a informação que necessitamos dos resultados do blast.
      - uniprot_id
      - evalue
      - score
      - identity
    """

    blast_results = {}

    for tag in tag_to_files:
        (_, out_file) = tag_to_files[tag]

        ### Alguns resultados do blast, ao serem lidos de disco
        ### na função rw.read_blast
        ### estavam a dar erro ao fazer parsing nesta linha.
        ### Como não necessitamos deste resultado,
        ### optamos por removê-la.
        regex = ".*Hsp_midline.*"
        rw.filter_file(regex, out_file)

        handle = rw.read_blast(out_file)

        result = []

        for a in handle.alignments:
            # extrair o uniprot id
            if type in ["local", "docker"]:
                # nos blast locais, o uniprot_id está no hit_def
                uniprot_id = a.hit_def.split("|")[1]
            elif type == "expasy":
                # nos blast expasy, o uniprot_id está no hit_id
                uniprot_id = a.hit_id.split("|")[1]

            # escolher sempre o primeiro hsp
            hsp = a.hsps[0]
            evalue = hsp.expect
            score = hsp.score
            identities = hsp.identities
            align_length = hsp.align_length
            identity = (identities * 100) / align_length

            hit = {}
            hit["uniprot_id"] = uniprot_id
            hit["evalue"] = evalue
            hit["score"] = score
            hit["identity"] = identity
            result.append(hit)

        blast_results[tag] = result

    return blast_results
```

<a name="fetch_uniprots"></a>
Nesta fase ainda precisávamos de mais informação além daquela que é possível extrair dos resultados do BLAST.
Utilizamos o serviço extração de informação em _batch_ disponibilizado pela UniProt (mais informação [aqui](http://www.uniprot.org/help/programmatic_access)), e criámos uma função que dada uma lista de __uniprot_id__ retorna um dicionário que mapeia estes id para um outro dicionário com a informação desejada. Esta função pode ser encontrada no módulo [`util.www`](https://github.com/vitorenesduarte/bionella/blob/master/util/www.py).

```python
def fetch_uniprots(ids):
    """
    Faz download da informação presente no site uniprot
    em formato xml dada uma lista de ids uniprot.
    """

    url = "http://www.uniprot.org/uploadlists/"
    max_retries = 100

    # Cada pedido à uniprot vai no máximo com 1000 ids.
    queries  = [ids[i:i+1000] for i in range(0, len(ids), 1000)]

    files = []

    for query in queries:
        query_all = " ".join(query)
        md5 = util.md5(query_all)

        data = {
            "format": "xml",
            "from": "ID",
            "to": "ACC",
            "uploadQuery": query_all,
            "jobId": md5
        }

        done = False
        attempt = 0

        # enquanto não acabar ou não esgotar as tentativas todas
        while not done and attempt < max_retries:

            # HTTP POST
            response = requests.post(
                url,
                data=data
            )

            if response.status_code == 200:
                # Gravar o xml
                file_path = "/tmp/" + md5 + ".xml"
                rw.write_file(response.text, file_path)
                files.append(file_path)
                done = True

            attempt += 1

        if not done:
            print("Não consegui fazer download de " + str(query))

    infos = {}

    for file_path in files:
        tree = parse_xml(file_path, add_root=True, start=2, end=1)
        entries = tree.findall(".//entry")

        for entry in entries:
            (uniprot_id, info) = extract_uniprot_info(entry)
            infos[uniprot_id] = info

    return infos
```


<a name="extract_uniprot_info"></a>
Podemos ver que esta função utiliza uma outra função `extract_uniprot_info`, também definida no módulo [`util.www`](https://github.com/vitorenesduarte/bionella/blob/master/util/www.py).

```python
# FINISH THIS ONCE WE DECIDE ALL THE INFO WE WANT FROM UNIPROT
def extract_uniprot_info(entry):
    """
    Extrai a informação que necessitamos da uniprot.
      - status (reviewed, unreviewed)
      - accessions
      - comentários sobre a função
      - GO - Molecular Function
      - sequência
    """
    # dataset
    ds = entry.get("dataset")

    if ds == "Swiss-Prot":
        status = "reviewed"
    elif ds == "TrEMBL":
        status = "unreviewed"
    else:
        print("Não conheço o dataset " + ds)

    # ...
```

<a name="main"></a>
Todos estes passos podem ser encontrados no ficheiro [second.py](https://github.com/vitorenesduarte/bionella/tree/master/second.py) e resumem-se a:

```python
import util.rw as rw
import util.blast as blast
import util.www as www

def get_tags_and_proteins(dictionary):
    """
    Esta função retorna uma lista de pares
    em que a primeira componente é a tag
    e a segunda a sequência de aminoácidos.
    """

    tags_and_proteins = []

    for tag in dictionary:
        if dictionary[tag]["type"] == "mRNA":
            # se for do tipo "mRNA" estamos na presença de uma proteína
            protein = dictionary[tag]["translation"]
            both = (tag, protein)
            tags_and_proteins.append(both)

    return sorted(tags_and_proteins, key=lambda tp : tp[0])

def add_info_to_blast_results(blast_results, uniprots):
    """
    Esta função adiciona aos blast results informação
    retirada do site da uniprot.
    """

    for tag in blast_results:
        for i in range(len(blast_results[tag])):
            uniprot_id = blast_results[tag][i]["uniprot_id"]

            # esta proteína aparece nos resultados do blast como
            # Q8TC84-2 mas a uniprot retorna a sua informação
            # como Q8TC84
            if uniprot_id == "Q8TC84-2":
                uniprot_id = "Q8TC84"

            properties = [
                "molecular_functions",
                "sequence",
                "status"
            ]

            for p in properties:
                value = uniprots[uniprot_id][p]
                blast_results[tag][i][p] = value

    return blast_results

def main():
    ncbi_json_path = ".ncbi.json"
    blast_results_json_path = ".blast_results.json"

    dictionary = rw.read_json(ncbi_json_path)
    tags_and_proteins = get_tags_and_proteins(dictionary)

    blast_results = blast.blastp(
        tags_and_proteins,
        "expasy"
    )

    uniprot_ids = set()
    for tag in blast_results:
        for result in blast_results[tag]:
            uniprot_ids.add(result["uniprot_id"])

    uniprot_ids = list(uniprot_ids)
    uniprots = www.fetch_uniprots(uniprot_ids)

    blast_results = add_info_to_blast_results(
        blast_results,
        uniprots
    )
    rw.write_json(blast_results, blast_results_json_path)
    
if __name__ == "__main__":
    main()
```